In [ ]:
import pandas as pd
import json
import networkx
import time

In [3]:
W = pd.read_csv("../data/dual-chart.csv")
W.set_index("Attacking", inplace = True)
nodes = W.index.to_list()

In [4]:
W_pruned = W.copy()

threshold = 1.5

for A in nodes:
    for B in nodes:
        if A == B:
            continue

        difference = abs(W.loc[A,B]-W.loc[B,A])

        if difference < threshold:

            W_pruned.loc[A,B] = 0
            W_pruned.loc[B,A] = 0

In [5]:
G_unpruned = networkx.from_pandas_adjacency(W)
G = networkx.from_pandas_adjacency(W_pruned)

triangles_unpruned = sum(networkx.triangles(G_unpruned).values())/3
triangles_pruned = sum(networkx.triangles(G).values())/3

print(f"Unpruned Triangles : {triangles_unpruned}\nPruned Triangles : {triangles_pruned}")

Unpruned Triangles : 585276.0
Pruned Triangles : 870.0


In [6]:
triangles = set()

for u in G:
    for v in G.neighbors(u):

        if v <= u:
            continue
        
        for w in G.neighbors(v):

            if w <= v:
                continue
            
            if G.has_edge(u,w):
                triangles.add((u,v,w))

triangles = list(triangles)

In [9]:
with open("../data/triangles.json", "w") as file:
    json.dump(triangles, file, indent = 2)